In [24]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import seaborn as sns
# from notebooks.User_Engagement_Analysis.ipynb import user_metrics


In [25]:
# Load dataset
df = pd.read_csv(r'..\data\Week1_challenge_data_source(CSV).csv')


# Assign Engagement and Experience Scores
We will compute the engagement score and experience score using the Euclidean distance between each user and the less engaged/worst experience cluster (from the k-means clustering results):

In [26]:
# Compute the average throughput (Download + Upload)
df['Avg_Throughput'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2

# Compute the average TCP retransmission (Download + Upload)
df['Avg_TCP_Retrans'] = (df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']) / 2

# Grouping by 'Handset Type' to analyze average throughput and TCP retransmission
throughput_by_handset = df.groupby('Handset Type')['Avg_Throughput'].mean()
tcp_retrans_by_handset = df.groupby('Handset Type')['Avg_TCP_Retrans'].mean()

# Display results
print(throughput_by_handset)
print(tcp_retrans_by_handset)

Handset Type
A-Link Telecom I. Cubot A5                                                             11755.000000
A-Link Telecom I. Cubot Note Plus                                                       3349.500000
A-Link Telecom I. Cubot Note S                                                          4468.500000
A-Link Telecom I. Cubot Nova                                                           28108.500000
A-Link Telecom I. Cubot Power                                                          34734.000000
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809       29.000000
Zyxel Communicat. Lte7460                                                              30978.000000
Zyxel Communicat. Sbg3600                                                              48675.500000
Zyxel Communicat. Zyxel Wah7706                                                        

In [27]:
# Top 10 Handset Types by Average Throughput
top_10_throughput = throughput_by_handset.nlargest(10)

# Top 10 Handset Types by Average TCP Retransmission
top_10_tcp_retrans = tcp_retrans_by_handset.nlargest(10)

# Display top handsets
print("Top 10 Handset Types by Throughput:")
print(top_10_throughput)

print("Top 10 Handset Types by TCP Retransmission:")
print(top_10_tcp_retrans)

Top 10 Handset Types by Throughput:
Handset Type
Huawei B715S-23C               54497.115385
New-Bund Technol. Mix 2        52605.500000
Lg G6+                         50067.500000
Zyxel Communicat. Sbg3600      48675.500000
Huawei Y9 2019                 45869.500000
Xiaomi Communica. M1902F1G     42806.833333
Spa Condor Elect. Allure M2    42155.750000
Ovvi-Cellphone T. Echo         39476.500000
Huawei Pele                    39190.500000
Spa Condor Elect. Plume L2     38264.500000
Name: Avg_Throughput, dtype: float64
Top 10 Handset Types by TCP Retransmission:
Handset Type
Apple iPad Pro (A1652)                       3.507656e+08
Huawei Bln-Al10                              1.645873e+08
Asustek Asus Zenfone Selfie Zd551Kl          1.608292e+08
Dynamic Tech Hol. D-Mobile I3 I5 I7 I8 I9    1.341563e+08
Samsung Galaxy Tab S3 (Sm-T825)              1.266421e+08
Huawei E5776S-32                             1.175107e+08
Samsung Galaxy J5                            8.481032e+07
Samsung Gal

In [28]:
# Option 1: Fill missing values with the column mean or median (or another suitable value)
df['Avg_Throughput'].fillna(df['Avg_Throughput'].mean(), inplace=True)
df['Avg_TCP_Retrans'].fillna(df['Avg_TCP_Retrans'].mean(), inplace=True)

C:\Users\Hello\AppData\Local\Temp\ipykernel_14696\2218360153.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Avg_Throughput'].fillna(df['Avg_Throughput'].mean(), inplace=True)
C:\Users\Hello\AppData\Local\Temp\ipykernel_14696\2218360153.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

In [29]:
# Select relevant columns (throughput and retransmission) for clustering
X = df[['Avg_Throughput', 'Avg_TCP_Retrans']].values

In [30]:
# Apply k-means clustering (k=2)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

# Add the cluster labels to the dataframe
df['Cluster'] = kmeans.labels_

In [31]:
# Get cluster centers (these represent the centroid of each cluster)
cluster_centers = kmeans.cluster_centers_

# Identify the cluster with lower throughput and retransmission (less engaged cluster)
low_throughput_value = cluster_centers[:, 0].min()  # Low throughput value
low_tcp_retrans_value = cluster_centers[:, 1].min()  # Low TCP retransmission value

# Identify the cluster with higher throughput and retransmission (highly engaged cluster)
high_throughput_value = cluster_centers[:, 0].max()  # high throughput value
high_tcp_retrans_value = cluster_centers[:, 1].max()  # high TCP retransmission value

In [32]:
print(f"Low Throughput Value: {low_throughput_value}")
print(f"Low TCP Retrans Value: {low_tcp_retrans_value}")

print(f"High Throughput Value: {high_throughput_value}")
print(f"High TCP Retrans Value: {high_tcp_retrans_value}")

Low Throughput Value: 7487.679343821968
Low TCP Retrans Value: 10706636.277198281
High Throughput Value: 35908.95019920318
High TCP Retrans Value: 1296679486.8605595


# Assign Engagement and Experience Scores Using the Identified Cluster Centers
Once you have the low_throughput_value and low_tcp_retrans_value, use them to calculate the engagement score as the Euclidean distance from these low values.

In [33]:
from scipy.spatial.distance import euclidean

# Example cluster center values (replace with your actual cluster centers)
less_engaged_cluster_center = [low_throughput_value, low_tcp_retrans_value]
worst_experience_cluster_center = [high_tcp_retrans_value, low_throughput_value]

# Calculate engagement score
df['Engagement_Score'] = df.apply(lambda row: euclidean(
    [row['Avg_Throughput'], row['Avg_TCP_Retrans']], less_engaged_cluster_center), axis=1)

# Calculate experience score
df['Experience_Score'] = df.apply(lambda row: euclidean(
    [row['Avg_Throughput'], row['Avg_TCP_Retrans']], worst_experience_cluster_center), axis=1)

# Display the engagement and experience scores
print(df[['Bearer Id','Engagement_Score', 'Experience_Score']].head())

      Bearer Id  Engagement_Score  Experience_Score
0  1.311448e+19      2.151860e+06      1.296743e+09
1  1.311448e+19      2.151860e+06      1.296743e+09
2  1.311448e+19      2.151860e+06      1.296743e+09
3  1.311448e+19      2.151860e+06      1.296743e+09
4  1.311448e+19      2.151860e+06      1.296743e+09


In [36]:
# Get cluster centers (representing the centroids of each cluster)
cluster_centers = kmeans.cluster_centers_

In [37]:
# Assume worst experience is characterized by low throughput and high TCP retransmission
worst_experience_cluster = np.argmax(cluster_centers[:, 1])

In [38]:
# Calculate Euclidean distance (experience score) between each user and the worst experience cluster
worst_cluster_center = cluster_centers[worst_experience_cluster]
df['Experience_Score'] = np.linalg.norm(X - worst_cluster_center, axis=1)

In [41]:
worst_cluster_center

array([3.59089502e+04, 1.29667949e+09])